In [1]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'/Users/dennis/Documents/myGit/main/myUtils')

In [2]:
# Generate synthetic data
import genData
import optDatasetRd
import optModelRd

num_data = 100
num_items = 5
num_features = 10

x, r, c, Q = genData.genDataRd(num_data,num_features, num_items,Q = 100)
alpha = 0.5

In [3]:
# test optmodel.py
optModel = optModelRd.optModel(x,r,c,Q,alpha)

In [20]:
res = optModel.solve(closed=True)

In [21]:
res2 = optModel.solve()[1]

In [23]:
res2

[13.733172107070164,
 13.188266792257654,
 9.749175005931729,
 13.136900370287725,
 12.435678755157786,
 14.01118353813948,
 11.889788641263427,
 14.797585992546617,
 12.68423242044391,
 11.093471320562674,
 16.6938027783911,
 14.532591738760296,
 14.277382251098272,
 12.390018931491845,
 14.219147412881792,
 11.280640859961059,
 13.65827672756878,
 8.793873339721458,
 14.985103313989157,
 13.650986355487507,
 9.771000861611013,
 12.67794587794971,
 14.959219874813822,
 16.107742834432425,
 16.354935252529863,
 12.403187436005613,
 12.886213851096384,
 13.705419557076667,
 12.295522659233376,
 16.350784695322705,
 11.132409047987673,
 13.442456256429834,
 14.390786938758165,
 12.295473953980053,
 14.930429518063706,
 14.820664979491488,
 13.21497226886368,
 16.16369959979311,
 14.619388532562304,
 9.574433786726829,
 14.11665805125499,
 14.886518153420067,
 13.724357667510892,
 13.955305287331427,
 11.404973506391334,
 13.631947570444138,
 14.18418137150907,
 15.032355613616467,
 11.60

In [24]:
res

131.89092996534322

In [31]:
from torch.utils.data import Dataset

class optDataset(Dataset):
    def __init__(self,x,r,c,Q,alpha):
        self.x = x
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data, self.num_items, self.num_features = x.shape

        self.opt_sol = []
        self.opt_obj = []
        self.opt_sol_c = []
        self.opt_obj_c = []

        self._solve()


    def _solve(self,closed=False):
        model = optModel(
            self.x,
            self.r,
            self.c,
            self.Q,
            self.alpha
        )
        if closed:
            sol, obj = model.solve(closed=True)
            self.opt_sol_c.append(sol.reshape(-1,self.num_items))
            self.opt_obj_c.append(obj)
        else:
            sol, obj = model.solve()
            self.opt_sol.append(sol.reshape(-1,self.num_items))
            self.opt_obj.append(obj)

        

        
        return self.opt_sol_c, self.opt_obj_c


In [32]:
dataset = optDataset(x,r,c,Q,alpha)

([], [])